我希望預測出新的會員 那些可能是流失群 那些不是

In [1]:
import numpy as np
import pandas as pd

In [2]:
kkboxdata2017=pd.read_csv('kkboxdata2017v2.csv',parse_dates=['registration_init_time'])
kkboxdata2017.head()

,Unnamed: 0,msno,city,bd,gender,registered_via,registration_init_time,"('num_25', 'mean')","('num_50', 'mean')","('num_75', 'mean')",...,"('total_secs', 'mean')",play_days,payment_method_id_mode,payment_plan_days_mode,plan_list_price_sum,actual_amount_paid_sum,is_auto_renew_sum,is_cancel_sum,transactions_times,is_churn
0,0,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,0,NaN,7,2011-09-14,5.258621,1.017241,0.827586,...,7163.224276,58,41,30,258,258,2,0,2,0
1,1,I0yFvqMoNkM8ZNHb617e1RBzIS/YRKemHO7Wj13EtA0=,13,63,male,9,2011-09-18,1.384615,0.576923,0.538462,...,4270.220462,26,40,30,298,298,2,0,2,0
2,2,OoDwiKZM+ZGr9P3fRivavgOtglTEaNfWJO4KaJcTTts=,1,0,NaN,7,2011-09-18,1.523810,0.285714,0.285714,...,3052.110524,21,41,30,298,298,2,0,2,0
3,3,4De1jAxNRABoyRBDZ82U0yEmzYkqeOugRGVNIf92Xb8=,4,28,female,9,2011-09-20,2.200000,0.400000,0.200000,...,3557.196400,5,36,30,360,360,2,0,2,0
4,4,Z6WIOK9vXy+e2XDBiioNAxuZ0ScXSU/Ebq4tUwqVSrE=,22,38,female,9,2011-09-29,1.800000,1.000000,0.700000,...,1662.418200,20,40,30,298,298,2,0,2,0


In [3]:
kkboxdata2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 770929 entries, 0 to 770928
Data columns (total 23 columns):
Unnamed: 0                770929 non-null int64
msno                      770929 non-null object
city                      770929 non-null int64
bd                        770929 non-null int64
gender                    356099 non-null object
registered_via            770929 non-null int64
registration_init_time    770929 non-null datetime64[ns]
('num_25', 'mean')        770929 non-null float64
('num_50', 'mean')        770929 non-null float64
('num_75', 'mean')        770929 non-null float64
('num_985', 'mean')       770929 non-null float64
('num_100', 'mean')       770929 non-null float64
('num_unq', 'mean')       770929 non-null float64
('total_secs', 'mean')    770929 non-null float64
play_days                 770929 non-null int64
payment_method_id_mode    770929 non-null int64
payment_plan_days_mode    770929 non-null int64
plan_list_price_sum       770929 non-null int64


因為類別太多 先做虛擬變數

In [4]:
dummiesc=['city','gender','registered_via','payment_method_id_mode']
kkboxdata2017 =pd.get_dummies(kkboxdata2017, prefix_sep="__",columns=dummiesc,dummy_na=True,drop_first=True)
cols = list(kkboxdata2017)
cols.pop(cols.index('is_churn'))
cols.append('is_churn')
kkboxdata2017 = kkboxdata2017.loc[:,cols]
kkboxdata2017.columns

Index(['Unnamed: 0', 'msno', 'bd', 'registration_init_time',
       '('num_25', 'mean')', '('num_50', 'mean')', '('num_75', 'mean')',
       '('num_985', 'mean')', '('num_100', 'mean')', '('num_unq', 'mean')',
       '('total_secs', 'mean')', 'play_days', 'payment_plan_days_mode',
       'plan_list_price_sum', 'actual_amount_paid_sum', 'is_auto_renew_sum',
       'is_cancel_sum', 'transactions_times', 'city__3.0', 'city__4.0',
       'city__5.0', 'city__6.0', 'city__7.0', 'city__8.0', 'city__9.0',
       'city__10.0', 'city__11.0', 'city__12.0', 'city__13.0', 'city__14.0',
       'city__15.0', 'city__16.0', 'city__17.0', 'city__18.0', 'city__19.0',
       'city__20.0', 'city__21.0', 'city__22.0', 'city__nan', 'gender__male',
       'gender__nan', 'registered_via__4.0', 'registered_via__7.0',
       'registered_via__9.0', 'registered_via__13.0', 'registered_via__nan',
       'payment_method_id_mode__11.0', 'payment_method_id_mode__14.0',
       'payment_method_id_mode__16.0', 'payment_m

訓練資料 設計在20161201之前註冊會員

In [5]:
traindata=kkboxdata2017.query('registration_init_time<20161202')
traindata['is_churn'].value_counts()

0    703552
1     36157
Name: is_churn, dtype: int64

驗證資料 設計在20161202與20170101之間註冊會員

In [6]:
validdata=kkboxdata2017.query('20170101>registration_init_time>20161201')
validdata['is_churn'].value_counts()

0    13920
1     1016
Name: is_churn, dtype: int64

測試資料 設計在20170101之後註冊會員

In [7]:
testdata=kkboxdata2017.query('registration_init_time>20170101')
testdata['is_churn'].value_counts()

0    13971
1     1857
Name: is_churn, dtype: int64

看結果而言 流失比率是少數 如果要找出預測流失客群 屬於二元分類問題 另外要對資料不平衡的情況做處理


先塞選好資料需要用的留

刪除不用的欄位

In [8]:
traindata.drop(['Unnamed: 0', 'msno','registration_init_time',], axis=1,inplace =True)
validdata.drop(['Unnamed: 0', 'msno','registration_init_time',], axis=1,inplace =True)
testdata.drop(['Unnamed: 0', 'msno','registration_init_time',], axis=1,inplace =True)

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


類別要使用虛擬變數

這邊要做分類的機器學習 這邊我先把主流的分類器跑一次 之後再挑選幾個作參數調整

In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import recall_score

In [10]:
dict_classifiers = {
    "Logistic Regression": LogisticRegression(),
    "Nearest Neighbors": KNeighborsClassifier(),
    #"rbf SVM": SVC(kernel='rbf',probability=True),
    #"linear SVM": SVC(kernel='linear',probability=True),
    #"poly SVM": SVC(kernel='poly',probability=True),
    #"sigmoid SVM": SVC(kernel='sigmoid',probability=True),
    "linear SVM sgd": SGDClassifier(loss='hinge'),
    "Gradient Boosting Classifier": GradientBoostingClassifier(n_estimators=100),
    "Decision Tree": tree.DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "Neural Net": MLPClassifier(alpha = 1),
    "Naive Bayes": GaussianNB(),
    "XGBoost": XGBClassifier()
    #"AdaBoost": AdaBoostClassifier(),
    #"QDA": QuadraticDiscriminantAnalysis(),
    #"Gaussian Process": GaussianProcessClassifier()
}

In [11]:
def batch_classify(X_train, Y_train, X_test, Y_test, no_classifiers = 5, verbose = True):
    """
    This method, takes as input the X, Y matrices of the Train and Test set.
    And fits them on all of the Classifiers specified in the dict_classifier.
    The trained models, and accuracies are saved in a dictionary. The reason to use a dictionary
    is because it is very easy to save the whole dictionary with the pickle module.
    
    Usually, the SVM, Random Forest and Gradient Boosting Classifier take quiet some time to train. 
    So it is best to train them on a smaller dataset first and 
    decide whether you want to comment them out or not based on the test accuracy score.
    """
    
    dict_models = {}
    for classifier_name, classifier in list(dict_classifiers.items())[:no_classifiers]:
        t_start = time.perf_counter()
        classifier.fit(X_train, Y_train)
        t_end = time.perf_counter()
        
        t_diff = t_end - t_start
        train_score = classifier.score(X_train, Y_train)
        test_score = classifier.score(X_test, Y_test)
        test_recall=recall_score(Y_test,classifier.predict(X_test))
        
        dict_models[classifier_name] = {'model': classifier, 'train_score': train_score, 'test_score': test_score, 'recall': test_recall, 'train_time': t_diff}
        if verbose:
            print("trained {c} in {f:.2f} s".format(c=classifier_name, f=t_diff))
    return dict_models



def display_dict_models(dict_models, sort_by='test_score'):
    cls = [key for key in dict_models.keys()]
    test_s = [dict_models[key]['test_score'] for key in cls]
    training_s = [dict_models[key]['train_score'] for key in cls]
    training_r = [dict_models[key]['recall'] for key in cls]
    training_t = [dict_models[key]['train_time'] for key in cls]
    
    df_ = pd.DataFrame(data=np.zeros(shape=(len(cls),5)), columns = ['classifier', 'train_score', 'test_score', 'recall', 'train_time'])
    for ii in range(0,len(cls)):
        df_.loc[ii, 'classifier'] = cls[ii]
        df_.loc[ii, 'train_score'] = training_s[ii]
        df_.loc[ii, 'test_score'] = test_s[ii]
        df_.loc[ii, 'recall'] = training_r[ii]
        df_.loc[ii, 'train_time'] = training_t[ii]
    
    display(df_.sort_values(by=sort_by, ascending=False))

In [12]:
dict_models = batch_classify(traindata.iloc[:,:-1], traindata.iloc[:,-1], validdata.iloc[:,:-1], validdata.iloc[:,-1], no_classifiers = 12)
display_dict_models(dict_models)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


trained Logistic Regression in 52.13 s
trained Nearest Neighbors in 36.06 s


C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


trained linear SVM sgd in 1.86 s
trained Gradient Boosting Classifier in 199.05 s
trained Decision Tree in 20.69 s
trained Random Forest in 200.95 s
trained Neural Net in 289.57 s
trained Naive Bayes in 1.43 s
trained XGBoost in 128.21 s


,classifier,train_score,test_score,recall,train_time
3,Gradient Boosting Classifier,0.972431,0.963511,0.630906,199.052585
5,Random Forest,0.999996,0.963243,0.630906,200.946515
8,XGBoost,0.972413,0.963042,0.628937,128.206317
4,Decision Tree,1.000000,0.947442,0.552165,20.687276
0,Logistic Regression,0.954102,0.939877,0.143701,52.129670
1,Nearest Neighbors,0.957191,0.938337,0.181102,36.055290
6,Neural Net,0.952453,0.936395,0.073819,289.566755
2,linear SVM sgd,0.950048,0.930102,0.004921,1.861573
7,Naive Bayes,0.908902,0.880758,0.777559,1.426174


因為資料偏頗 測試分數相對較高 因此可以參考recall 算出對流失值的正確比例 越高代表對流失判斷能力越高

因此前三名為較適合模型 

若再考慮成本下 Naive Bayes的recall是裡面高 也是可以考慮的模型 且訓練時間少很多

再來利用 隨機森林幫助我們先挑選可能比較有用的變數

In [22]:
feat_labels = traindata.columns[:-1]

forest = RandomForestClassifier(n_estimators=500,
                                random_state=1)

forest.fit(traindata.iloc[:,:-1], traindata.iloc[:,-1])
importances = forest.feature_importances_

indices = np.argsort(importances)[::-1]
for f in range(traindata.shape[1]-1):
    print("%2d) %-*s %f" % (f + 1, 30,feat_labels[indices[f]],importances[indices[f]]))

 1) is_cancel_sum                  0.181322
 2) is_auto_renew_sum              0.088585
 3) play_days                      0.073027
 4) plan_list_price_sum            0.062124
 5) actual_amount_paid_sum         0.059573
 6) ('total_secs', 'mean')         0.048914
 7) ('num_100', 'mean')            0.048175
 8) ('num_unq', 'mean')            0.048030
 9) ('num_25', 'mean')             0.047532
10) ('num_50', 'mean')             0.046299
11) ('num_985', 'mean')            0.046163
12) ('num_75', 'mean')             0.045549
13) transactions_times             0.041366
14) payment_method_id_mode__38.0   0.031286
15) bd                             0.029198
16) payment_method_id_mode__39.0   0.014181
17) registered_via__9.0            0.006175
18) gender__male                   0.005970
19) payment_method_id_mode__41.0   0.005883
20) registered_via__7.0            0.005743
21) registered_via__4.0            0.005475
22) city__13.0                     0.004713
23) payment_method_id_mode__36.0

挑選比重比較高的前21個再做訓練

In [31]:
dict_models = batch_classify(traindata.loc[:,feat_labels[indices[0:21]]], traindata.iloc[:,-1], validdata.loc[:,feat_labels[indices[0:21]]], validdata.iloc[:,-1], no_classifiers = 12)
display_dict_models(dict_models)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


trained Logistic Regression in 36.21 s
trained Nearest Neighbors in 24.43 s


C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


trained linear SVM sgd in 1.20 s
trained Gradient Boosting Classifier in 120.88 s
trained Decision Tree in 13.80 s
trained Random Forest in 200.12 s
trained Neural Net in 157.68 s
trained Naive Bayes in 0.48 s
trained XGBoost in 55.59 s


,classifier,train_score,test_score,recall,train_time
8,XGBoost,0.972361,0.963109,0.630906,55.591392
3,Gradient Boosting Classifier,0.972392,0.962908,0.633858,120.883721
5,Random Forest,0.999991,0.962908,0.627953,200.121076
0,Logistic Regression,0.966649,0.956548,0.494094,36.214903
4,Decision Tree,1.000000,0.946505,0.541339,13.802442
1,Nearest Neighbors,0.957195,0.937734,0.172244,24.433297
6,Neural Net,0.952279,0.934387,0.045276,157.683388
2,linear SVM sgd,0.950817,0.932043,0.001969,1.202075
7,Naive Bayes,0.895536,0.878682,0.825787,0.484603


結果發現大部分差不多 而訓練時間大幅降低

且Logistic Regression 跟 Naive Bayes 的 recall 也有明顯提升 對我們要加強找出少數的流失值有幫助

利用SVM做分類 但是SVM數據量太多會跑很久 必須降維跟抽樣減少維度

In [35]:
svc0=SVC(kernel='rbf',probability=True).fit(traindata.loc[:,feat_labels[indices[0:3]]], traindata.iloc[:,-1])

C:\Users\user\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [37]:
svc0.score(traindata.loc[:,feat_labels[indices[0:3]]], traindata.iloc[:,-1])

0.9638006297070875

In [38]:
svc0.score(validdata.loc[:,feat_labels[indices[0:3]]], validdata.iloc[:,-1])

0.9482458489555436

In [39]:
recall_score(validdata.iloc[:,-1],svc0.predict(validdata.loc[:,feat_labels[indices[0:3]]]))

0.3188976377952756

In [56]:
traindatasample=traindata.sample(50000)
validdatasample=validdata.sample(10000)

In [57]:
%%time
svc1=SVC(kernel='rbf',probability=True).fit(traindatasample.loc[:,feat_labels[indices[0:3]]], traindatasample.iloc[:,-1])
print(svc1.score(traindatasample.loc[:,feat_labels[indices[0:3]]], traindatasample.iloc[:,-1]))
print(svc1.score(validdatasample.loc[:,feat_labels[indices[0:3]]], validdatasample.iloc[:,-1]))
print(recall_score(validdatasample.iloc[:,-1],svc1.predict(validdatasample.loc[:,feat_labels[indices[0:3]]])))

C:\Users\user\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.96546
0.9469
0.2743362831858407
Wall time: 38.7 s


SVM結果表現一般不算是適合模型